In [113]:
# importlar(imports)
import findspark
import pandas as pd
from multipledispatch import dispatch
import time
import binascii
from pyDes import des, CBC, PAD_PKCS5

In [114]:
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('Tez').getOrCreate()

**Importing pySpark Data types**

In [115]:
from pyspark.sql.types import (StructField, StringType, StructType, IntegerType, FloatType)

In [ ]:
# column isimleri

data_schema = [
    StructField('age', IntegerType(), True),
    StructField('workclass', StringType(), True),
    StructField('fnlwgt', FloatType(), True),
    StructField('education', StringType(), True),
    StructField('education-num', FloatType(), True),
    StructField('marital-status', StringType(), True),
    StructField('occupation', StringType(), True),
    StructField('relationship', StringType(), True),
    StructField('race', StringType(), True),
    StructField('sex', StringType(), True),
    StructField('capital-gain', FloatType(), True),
    StructField('capital-loss', FloatType(), True),
    StructField('hours-per-week', FloatType(), True),
    StructField('native-country', StringType(), True),
    StructField('income', StringType(), True),
]

final_struct = StructType(fields=data_schema)

df = spark.read.csv("./DATASET/adult.all.txt", sep=",", header=None, schema=final_struct)

**Here's how Dataset Looks like**

In [117]:
df.show()
df.printSchema()

+---+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
|age|        workclass|  fnlwgt|    education|education-num|      marital-status|        occupation|  relationship|               race|    sex|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516.0|    Bachelors|         13.0|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|      2174.0|         0.0|          40.0| United-States| <=50k|
| 50| Self-emp-not-inc| 83311.0|    Bachelors|         13.0|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|         0.0|         0.0|   

**Making the tables user & graphic friendly -> Converting to Pandas**

In [118]:
# PANDAYA DÖnüşüm
df = df.toPandas()

In [119]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50k
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50k
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50k
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50k
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50k


In [120]:

weights = {}
print(df["age"])
for col in df.columns:
  weights[col] = len(df[col].unique())

sortedWt = dict(sorted(weights.items(), key=lambda item: item[1], reverse=True))

n = len(sortedWt.keys())
for key in sortedWt.keys():
  sortedWt[key] = n
  n = n - 1


0        39
1        50
2        38
3        53
4        28
         ..
48837    39
48838    64
48839    38
48840    44
48841    35
Name: age, Length: 48842, dtype: int32


In [121]:
sortedWt

{'fnlwgt': 15,
 'capital-gain': 14,
 'capital-loss': 13,
 'hours-per-week': 12,
 'age': 11,
 'native-country': 10,
 'education': 9,
 'education-num': 8,
 'occupation': 7,
 'workclass': 6,
 'marital-status': 5,
 'relationship': 4,
 'race': 3,
 'sex': 2,
 'income': 1}

In [122]:
def encrypt(secret_key,s):
    start = time.time()
    iv = secret_key
    k = des(secret_key, CBC, iv, pad=None, padmode=PAD_PKCS5)
    en = k.encrypt(s, padmode=PAD_PKCS5)
    t = (time.time() - start)*1000
    return { 'data': binascii.b2a_hex(en), 'time': t }

def decrypt(secret_key, s):
    iv = secret_key
    k = des(secret_key, CBC, iv, pad=None, padmode=PAD_PKCS5)
    de = k.decrypt(binascii.a2b_hex(s), padmode=PAD_PKCS5)
    return de

In [123]:

et = {}
key = "letsdoit"
for col in df.columns:
  et[col] = {}

for col in df.columns:
  for row in df[col]:
    if row not in et[col].keys():
      et[col][row] = encrypt(key, str(row))

In [124]:
print(et["age"])

{39: {'data': b'f4d147fc2ec6786c', 'time': 1.0809898376464844}, 50: {'data': b'fb7d6438d20ec08c', 'time': 0.0}, 38: {'data': b'ee48689b4efb2070', 'time': 0.0}, 53: {'data': b'0f63db7f3b4ad980', 'time': 1.3036727905273438}, 28: {'data': b'9916cf68a3834cc8', 'time': 0.0}, 37: {'data': b'183228d7b4a53cf0', 'time': 0.0}, 49: {'data': b'08b2bbef1138e43a', 'time': 1.094818115234375}, 52: {'data': b'85d8875585416279', 'time': 0.0}, 31: {'data': b'da01539f23a8f933', 'time': 0.0}, 42: {'data': b'40c3a71fee1464e3', 'time': 1.0225772857666016}, 30: {'data': b'a36ed599ead97ff6', 'time': 0.0}, 23: {'data': b'459cc7fea1d43a6f', 'time': 0.0}, 32: {'data': b'649e0aaeb1dc037d', 'time': 1.0216236114501953}, 40: {'data': b'9578617171dd6c03', 'time': 0.0}, 34: {'data': b'c9b20ede1b3b8782', 'time': 0.0}, 25: {'data': b'9d79980d40a2fe27', 'time': 1.363515853881836}, 43: {'data': b'7a1a9a0c5f151855', 'time': 0.0}, 54: {'data': b'0cdf0a37a9604f6f', 'time': 1.0013580322265625}, 35: {'data': b'37fece6d6b0c5365'

**Generating a Helper Function**<br>Calculating and storing time of execution and uniqueness count for a single dataentry of every attribute

In [125]:
nt = {}
for col in df.columns:
  nt[col] = { 'count': len(df[col].unique()), 'time': encrypt(key, str(df[col][0]))['time']}

In [126]:
nt

{'age': {'count': 74, 'time': 0.99945068359375},
 'workclass': {'count': 9, 'time': 1.001119613647461},
 'fnlwgt': {'count': 28523, 'time': 0.9996891021728516},
 'education': {'count': 16, 'time': 1.0001659393310547},
 'education-num': {'count': 16, 'time': 0.9999275207519531},
 'marital-status': {'count': 7, 'time': 1.0027885437011719},
 'occupation': {'count': 15, 'time': 0.9999275207519531},
 'relationship': {'count': 6, 'time': 1.0485649108886719},
 'race': {'count': 5, 'time': 1.0023117065429688},
 'sex': {'count': 2, 'time': 0.0},
 'capital-gain': {'count': 123, 'time': 0.0},
 'capital-loss': {'count': 99, 'time': 1.0001659393310547},
 'hours-per-week': {'count': 96, 'time': 0.0},
 'native-country': {'count': 42, 'time': 0.0},
 'income': {'count': 2, 'time': 0.0}}

In [127]:
helperEt = {}

for col in df.columns:
  start = time.time()
  df[col].sort_values(ascending = False)
  helperEt[col] = (time.time() - start)*1000
   

In [128]:
helperEt

{'age': 3.7734508514404297,
 'workclass': 43.86615753173828,
 'fnlwgt': 5.222082138061523,
 'education': 34.73687171936035,
 'education-num': 1.9998550415039062,
 'marital-status': 29.77156639099121,
 'occupation': 30.823945999145508,
 'relationship': 34.02376174926758,
 'race': 30.855417251586914,
 'sex': 32.03463554382324,
 'capital-gain': 2.145051956176758,
 'capital-loss': 1.0030269622802734,
 'hours-per-week': 2.001523971557617,
 'native-country': 34.35873985290527,
 'income': 30.04598617553711}

In [129]:
st = {}

for col in df.columns:
  st[col] = sortedWt[col]/helperEt[col]

sortedSt = dict(sorted(st.items(), key=lambda item: item[1], reverse=True))

In [130]:
print(sortedSt)

{'capital-loss': 12.960768243403852, 'capital-gain': 6.526648438368345, 'hours-per-week': 5.995431566408577, 'education-num': 4.000289938006676, 'age': 2.9151035572123587, 'fnlwgt': 2.872417477057937, 'native-country': 0.2910467625649673, 'education': 0.2590906882090915, 'occupation': 0.22709616738214022, 'marital-status': 0.1679454797350866, 'workclass': 0.13677970302411027, 'relationship': 0.11756489565960787, 'race': 0.09722765942650502, 'sex': 0.062432425593355316, 'income': 0.03328231578613258}


In [131]:

tc = 100000

for key in sortedSt.keys():
  idx = 0
  for el in df[key]:
    if tc >= et[key][el]['time']:
      df.loc[idx, key] = et[key][el]['data']
      tc = tc - et[key][el]['time']
    idx = idx + 1

In [134]:
# Let's take a look at resulting dataset
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,b'f4d147fc2ec6786c',State-gov,b'f5b93b162b5e383f',Bachelors,b'27ddc835a5acb313',Never-married,Adm-clerical,b'5604151d01d05203170950cdb9c25299',b'c1169a1459425ffa',b'd75fd9147dfd620d',b'8323049c46985b1f',b'0ed705ec7fac3627',b'aa94490952c42c39',b'4fc963bd4ff11fbb4d7039c15e6cb62b',b'52c3982724b290d8'
1,b'fb7d6438d20ec08c',Self-emp-not-inc,b'9dff9501c50020c1',Bachelors,b'27ddc835a5acb313',b'f6b59e3caeb7a5fb44d9ac71729c2350b21f16dc63a4...,b'b9db003c6adba57dba178cdef5d94a93a987265a8da8...,Husband,b'c1169a1459425ffa',b'd75fd9147dfd620d',b'0ed705ec7fac3627',b'0ed705ec7fac3627',b'27ddc835a5acb313',b'4fc963bd4ff11fbb4d7039c15e6cb62b',b'52c3982724b290d8'
2,b'ee48689b4efb2070',Private,b'd11e7837093fb7265462b5120e3f9cc8',b'b19088fdc9c9758ad2fe5e6bc784510c',b'40faccc8d7265aae',Divorced,Handlers-cleaners,b'5604151d01d05203170950cdb9c25299',b'c1169a1459425ffa',b'd75fd9147dfd620d',b'0ed705ec7fac3627',b'0ed705ec7fac3627',b'aa94490952c42c39',b'4fc963bd4ff11fbb4d7039c15e6cb62b',b'52c3982724b290d8'
3,b'0f63db7f3b4ad980',Private,b'ed561535029b17e2ef9e6647c3d81189',11th,b'8e73d72ae467f5be',b'f6b59e3caeb7a5fb44d9ac71729c2350b21f16dc63a4...,Handlers-cleaners,Husband,Black,b'd75fd9147dfd620d',b'0ed705ec7fac3627',b'0ed705ec7fac3627',b'aa94490952c42c39',b'4fc963bd4ff11fbb4d7039c15e6cb62b',b'52c3982724b290d8'
4,b'9916cf68a3834cc8',Private,b'f66dfc57448f43d77832a2cc12861a9c',Bachelors,b'27ddc835a5acb313',b'f6b59e3caeb7a5fb44d9ac71729c2350b21f16dc63a4...,Prof-specialty,b'f36450d2f595c77c',Black,b'7416e193041af998',b'0ed705ec7fac3627',b'0ed705ec7fac3627',b'aa94490952c42c39',b'2b29bbdced3a8972',b'52c3982724b290d8'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,b'f4d147fc2ec6786c',Private,b'f37e2ed30d3a5f859477876243b54f7b',Bachelors,b'27ddc835a5acb313',Divorced,Prof-specialty,b'5604151d01d05203170950cdb9c25299',b'c1169a1459425ffa',b'7416e193041af998',b'0ed705ec7fac3627',b'0ed705ec7fac3627',b'c3e4150dc3fac58c',United-States,b'52c3982724b290d8'
48838,b'4fbb6d821e0c7bd2',?,b'4675a6ce30f07c3cee998bab15f5ceb1',b'b19088fdc9c9758ad2fe5e6bc784510c',b'40faccc8d7265aae',b'35cbe04c44c3a7c9e1b34b78a6ee4530',b'e1884bccc45f60cd',Other-relative,Black,b'd75fd9147dfd620d',b'0ed705ec7fac3627',b'0ed705ec7fac3627',b'aa94490952c42c39',United-States,b'52c3982724b290d8'
48839,b'ee48689b4efb2070',Private,b'ee601dc7d936a77cf514912dbc6274f7',Bachelors,b'27ddc835a5acb313',b'f6b59e3caeb7a5fb44d9ac71729c2350b21f16dc63a4...,Prof-specialty,Husband,b'c1169a1459425ffa',b'd75fd9147dfd620d',b'0ed705ec7fac3627',b'0ed705ec7fac3627',b'e28c1299ba354465',United-States,b'52c3982724b290d8'
48840,b'3d5b288bf08f6478',Private,b'f7612803b6e1399b',Bachelors,b'27ddc835a5acb313',Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,b'd75fd9147dfd620d',b'c20324bf42eb790e',b'0ed705ec7fac3627',b'aa94490952c42c39',United-States,b'52c3982724b290d8'
